In [3]:
import lib.models
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from sklearn.model_selection import train_test_split
import sys
import os
import notify2
notify2.init("Tesi.AI")

PROJECT_ROOT = os.path.abspath(os.getcwd() + os.sep + os.pardir)

sys.path.insert(0, PROJECT_ROOT)

from lib.time_series_datasets import *
from lib.models import *
from lib.utility import plot_matrices

PROJECT_NAME = "character trajectories"

DATA_DIR = os.path.join("data", PROJECT_NAME)
DATA_DIR = os.path.join(PROJECT_ROOT, DATA_DIR)

TUNER_DIR = os.path.join("models", "tuner", PROJECT_NAME)
TUNER_DIR = os.path.join(PROJECT_ROOT, TUNER_DIR)

IMAGE_DIR = "images"
IMAGE_DIR = os.path.join(PROJECT_ROOT, IMAGE_DIR)

TB_DIR = os.path.abspath(os.sep) + "tmp" + os.sep + "tensorboard" + os.sep

print("data dir: ", DATA_DIR)
print("tuner dir:", TUNER_DIR)
print("tensorboard dir:", TB_DIR)

EPOCHS = 200
GUESSES = 1
PATIENCE = 10
MAX_UNITS = 200

#set the following values based on the specific dataset
OUTPUT_UNITS = 20
OUTPUT_ACTIVATION = tf.keras.activations.softmax  # https://www.tensorflow.org/api_docs/python/tf/keras/activations  'softmax'
LOSS_FUNCTION =  tf.keras.losses.SparseCategoricalCrossentropy()  # https://www.tensorflow.org/api_docs/python/tf/keras/losses  'sparse_categorical_crossentropy'

if not os.path.exists(TUNER_DIR):
    os.makedirs(TUNER_DIR)
if not os.path.exists(TB_DIR):
    os.makedirs(TB_DIR)


2021-12-13 14:58:28.581755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-13 14:58:28.581784: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


data dir:  /dati/luca/Uni-Luca/Tesi/tesi/data/character trajectories
tuner dir: /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/character trajectories
tensorboard dir: /tmp/tensorboard/


In [5]:
x_test, y_test = load_sktime_dataset(os.path.join(DATA_DIR, 'test.ts'))
x_train_all, y_train_all = load_sktime_dataset(os.path.join(DATA_DIR, 'train.ts'))  # Max shape serve per avere il test set e il train set della stessa lunghezza

x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.33, random_state=42)

features = x_train.shape[-1]

train_set = (x_train, y_train)
val_set = (x_val, y_val)
test_set = (x_test, y_test)

benchmarks = {}
print("Train shape:\n\tinput: {}\n\toutput: {}\nTest shape:\n\tinput: {}\n\toutput: {}\n".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))


/dati/luca/Uni-Luca/Tesi/tesi/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(


Train shape:
	input: (952, 180, 3)
	output: (952,)
Test shape:
	input: (1436, 182, 3)
	output: (1436,)



In [20]:
# Type 1 ESN FullConnected

In [27]:
tf.random.set_seed(42)
model_name = 'ESN1 C1'

def build_model(hp):
    tmp_model = lib.models.ESN1_SLOW(units=hp.Int('units', min_value=5, max_value=MAX_UNITS),
                output_units=OUTPUT_UNITS,
                output_activation=OUTPUT_ACTIVATION,
                input_scaling=hp.Float('input_scaling', min_value=0.1, max_value=1.5, step=0.1),
                bias_scaling=hp.Float('bias_scaling', min_value=0.1, max_value=1.5, step=0.1),
                spectral_radius=hp.Float('spectral_radius', min_value=0.1, max_value=1.5, step=0.1),
                leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                )
    alpha = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

model = tune_and_test(model_name, build_model, train_set, val_set, test_set, EPOCHS, PATIENCE, GUESSES, benchmarks, tuner_path=TUNER_DIR, benchmarks_verbose=1)

INFO:tensorflow:Reloading Oracle from existing project /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/character trajectories/ESN1 C1 hyperband/oracle.json
USE bias: True
Dtype:  float32
INFO:tensorflow:Reloading Tuner from /dati/luca/Uni-Luca/Tesi/tesi/models/tuner/character trajectories/ESN1 C1 hyperband/tuner0.json
INFO:tensorflow:Oracle triggered exit
Start 1 benchmarks:
USE bias: True
Dtype:  float32
Epoch 1/200
30/30 [==============================] - 2s 49ms/step - loss: 2.8905 - accuracy: 0.2563 - val_loss: 1.9029 - val_accuracy: 0.4404
Epoch 2/200
29/30 [============================>.] - ETA: 0s - loss: 1.9080 - accuracy: 0.3955

KeyboardInterrupt: 

In [ ]:
units = model.units
print("units: ", units)

In [ ]:
tf.random.set_seed(42)
model_name = 'ESN2 C1'

def build_model(hp):
    tmp_model = ESN2(units=units,
                     sub_reservoirs=features,
                     output_units=OUTPUT_UNITS,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input_scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias_scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral_radius', min_value=0.1, max_value=1.5, step=0.1),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     )

    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(
            hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='log')),
        loss=LOSS_FUNCTION,
        metrics=['accuracy'])
    return tmp_model

model = tune_and_test(model_name, build_model, train_set, val_set, test_set, EPOCHS, PATIENCE, TUNER_DIR, GUESSES)

In [ ]:
tf.random.set_seed(42)
model_name = 'ESN3 C1'

def build_model(hp):
    connectivity = [[]]
    for i in range(features):
        for j in range(features):
            if i == j:
                connectivity[i][j] = 1
            else:
                connectivity[i][j] = hp.Float('conn '+str(i)+'-'+str(j), min_value=0.1, max_value=1)


    tmp_model = ESN3(units=units,
                     sub_reservoirs=features,
                     connectivity=connectivity,
                     output_units=OUTPUT_UNITS,
                     output_activation=OUTPUT_ACTIVATION,
                     input_scaling=hp.Float('input_scaling', min_value=0.1, max_value=1.5, step=0.1),
                     bias_scaling=hp.Float('bias_scaling', min_value=0.1, max_value=1.5, step=0.1),
                     spectral_radius=hp.Float('spectral_radius', min_value=0.1, max_value=1.5, step=0.1),
                     leaky=hp.Float('leaky', min_value=0.1, max_value=1, step=0.1),
                     )
    alpha = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.RMSprop(alpha),
        loss=LOSS_FUNCTION,
        metrics=['accuracy']
    )
    return tmp_model

model = tune_and_test(model_name, build_model, train_set, val_set, test_set, EPOCHS, PATIENCE, TUNER_DIR, GUESSES)

In [40]:
kernel_init = Kernel(initializer=tf.keras.initializers.RandomUniform(minval=-1., maxval=1))
recurrent_kernel_init = RecurrentFullConnected(0.4)
bias_init = tf.keras.initializers.RandomUniform(minval=-0.3, maxval=0.3)

model = keras.Sequential([
    keras.layers.Masking(),
    lib.esn.ESN(168, 0.5, kernel_initializer=kernel_init, recurrent_initializer=recurrent_kernel_init, bias_initializer=bias_init),
    keras.layers.Dense(20, activation=OUTPUT_ACTIVATION, name="readout")
    ])

model.compile(
        optimizer=keras.optimizers.RMSprop(0.5),
        loss=LOSS_FUNCTION,
        metrics=['accuracy']
    )

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=EPOCHS,
callbacks=keras.callbacks.EarlyStopping(monitor='val_loss', patience = 1, restore_best_weights = True))

print(model.trainable_weights)

Epoch 1/200
30/30 [==============================] - 2s 47ms/step - loss: 83.6373 - accuracy: 0.1492 - val_loss: 50.7271 - val_accuracy: 0.1319
Epoch 2/200
30/30 [==============================] - 1s 38ms/step - loss: 52.4721 - accuracy: 0.1712 - val_loss: 71.2787 - val_accuracy: 0.1426
[<tf.Variable 'readout/kernel:0' shape=(168, 20) dtype=float32, numpy=
array([[ 2.811297 ,  3.235911 ,  5.01888  , ...,  4.0138474,  3.9028633,
         2.975837 ],
       [ 2.5162125,  3.4989874,  4.780198 , ...,  3.6714506,  3.2605605,
         3.3053272],
       [ 2.6636488,  3.0830843,  2.9081244, ...,  3.110036 ,  4.6934934,
         3.5236776],
       ...,
       [ 2.6906118,  2.8970194,  1.7279671, ...,  4.140754 ,  3.9719844,
         3.0833235],
       [-2.7536223, -2.7324097, -3.03714  , ..., -4.267496 , -3.9416988,
        -3.0553265],
       [-2.8317165, -2.9242358, -1.444787 , ..., -3.7799802, -3.8905382,
        -3.0403194]], dtype=float32)>, <tf.Variable 'readout/bias:0' shape=(20,) dtype